## MLflow 5 minute Tracking Quickstart

This notebook demonstrates using a local MLflow Tracking Server to log, register, and then load a model as a generic Python Function (pyfunc) to perform inference on a Pandas DataFrame.

Throughout this notebook, we'll be using the MLflow fluent API to perform all interactions with the MLflow Tracking Server.

In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

import mlflow
from mlflow.models import infer_signature

### Set the MLflow Tracking URI 

Depending on where you are running this notebook, your configuration may vary for how you initialize the interface with the MLflow Tracking Server. 

For this example, we're using a locally running tracking server, but other options are available (The easiest is to use the free managed service within [Databricks Community Edition](https://community.cloud.databricks.com/)). 

Please see [the guide to running notebooks here](https://www.mlflow.org/docs/latest/getting-started/running-notebooks/index.html) for more information on setting the tracking server uri and configuring access to either managed or self-managed MLflow tracking servers.

In [2]:
# NOTE: review the links mentioned above for guidance on connecting to a managed tracking server, such as the free Databricks Community Edition

mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

In [3]:
cd ..

/workspaces/project7


/home/codespace/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


## Load training data and train a simple model

For our quickstart, we're going to be using the familiar iris dataset that is included in scikit-learn. Following the split of the data, we're going to train a simple logistic regression classifier on the training data and calculate some error metrics on our holdout test data. 

Note that the only MLflow-related activities in this portion are around the fact that we're using a `param` dictionary to supply our model's hyperparameters; this is to make logging these settings easier when we're ready to log our model and its associated metadata.

In [4]:
X_train = pd.read_csv("X_train_full.csv").set_index(keys=["SK_ID_CURR"])
X_test = pd.read_csv("X_test_full.csv").set_index(keys=["SK_ID_CURR"])
y_train = pd.read_csv("y_train.csv").set_index(keys=["Unnamed: 0"])
y_test = pd.read_csv("y_test.csv").set_index(keys=["Unnamed: 0"])


In [5]:

# Define the model hyperparameters
params = {'boosting_type': 'gbdt',
          'learning_rate': 0.1,
          'max_depth': -1,
          'num_leaves': 31,
          'class_weight' : 'balanced'}

# Train the model
lr = lgb.LGBMClassifier(**params)
lr.fit(X_train, y_train)

# Predict on the test set
y_pred = lr.predict(X_test)

# Calculate accuracy as a target loss metric
roc_auc_score = roc_auc_score(y_test, y_pred)

/home/codespace/.python/current/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/codespace/.python/current/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Info] Number of positive: 24732, number of negative: 281779
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.136457 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3938
[LightGBM] [Info] Number of data points in the train set: 306511, number of used features: 56
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000


## Define an MLflow Experiment

In order to group any distinct runs of a particular project or idea together, we can define an Experiment that will group each iteration (runs) together. 
Defining a unique name that is relevant to what we're working on helps with organization and reduces the amount of work (searching) to find our runs later on. 

In [6]:
mlflow.set_experiment("MLflow Quickstart")

<Experiment: artifact_location='mlflow-artifacts:/980107142953160213', creation_time=1711442171497, experiment_id='980107142953160213', last_update_time=1711442171497, lifecycle_stage='active', name='MLflow Quickstart', tags={}>

## Log the model, hyperparameters, and loss metrics to MLflow.

In order to record our model and the hyperparameters that were used when fitting the model, as well as the metrics associated with validating the fit model upon holdout data, we initiate a run context, as shown below. Within the scope of that context, any fluent API that we call (such as `mlflow.log_params()` or `mlflow.sklearn.log_model()`) will be associated and logged together to the same run. 

In [7]:
# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("roc_auc_score", roc_auc_score)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "ML LightGBM")

    # Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-quickstart",
    )

Registered model 'tracking-quickstart' already exists. Creating a new version of this model...
2024/03/27 07:29:39 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart, version 3
Created version '3' of model 'tracking-quickstart'.


## Load our saved model as a Python Function

Although we can load our model back as a native scikit-learn format with `mlflow.sklearn.load_model()`, below we are loading the model as a generic Python Function, which is how this model would be loaded for online model serving. We can still use the `pyfunc` representation for batch use cases, though, as is shown below.

In [8]:
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Use our model to predict the iris class type on a Pandas DataFrame

In [9]:
predictions = loaded_model.predict(X_test)

feature_names = X_test.columns

# Convert X_test validation feature data to a Pandas DataFrame
result = pd.DataFrame(X_test, columns=feature_names)

# Add the actual classes to the DataFrame
result["actual_class"] = y_test

# Add the model predictions to the DataFrame
result["predicted_class"] = predictions

result[:4]

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,...,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_8,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,actual_class,predicted_class
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
384575,0.0,1.0,1.0,0.0,6.0,1.0,4.0,1.0,1.0,14.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,1
214010,0.0,0.0,1.0,1.0,6.0,1.0,1.0,3.0,1.0,10.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0,0
142232,0.0,0.0,1.0,0.0,6.0,7.0,4.0,1.0,1.0,14.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0,1
389171,0.0,0.0,0.0,1.0,6.0,4.0,4.0,5.0,1.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.5,0,0
